In [1]:
import numpy as np
from startup import *
from math import *
import pylab as p
from matplotlib import *
from utils import *
#import pyfits
import astropy.io.fits as pyfits
import re
import numpy.random as rand
import random as random
#import random as randd
import pickle
import random as random
from scipy.interpolate import *
import csv
import sys
from astropy.table import Table
from matplotlib.colors import LogNorm
from matplotlib.ticker import MaxNLocator
from astropy.table import join
from astropy.coordinates import SkyCoord
import astropy.units as units
#--setfig --
def setfig(fig, **kwargs):
    if fig == 0 :
        pass
    elif fig:
        p.figure(fig,**kwargs)
        p.clf()
    else:
        p.figure(**kwargs)
#-------


HETDEX_path = '/Users/khawkins/Desktop/data/HETDEX/HETVIPS/'
waves = linspace(3470,5540,num=1036)

%matplotlib widget


In [5]:
import matplotlib
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 22}
matplotlib.rc('font', **font)

p.rcParams["xtick.direction"] = "in"
p.rcParams["ytick.direction"] = "in"
p.rcParams['font.size'] = 16
p.rcParams['xtick.top'] = True
p.rcParams['xtick.major.size']=6
p.rcParams['xtick.minor.size']=3
p.rcParams['xtick.major.width']=1.5#2.0
p.rcParams['xtick.minor.width']=1.0
p.rcParams['xtick.minor.visible'] = True

p.rcParams['font.size'] = 16
p.rcParams['ytick.right'] = True
p.rcParams['ytick.major.size']=6
p.rcParams['ytick.minor.size']=3
p.rcParams['ytick.major.width']=1.5#2.0
p.rcParams['ytick.minor.width']=1.0
p.rcParams['ytick.minor.visible'] = True


p.rc('axes',prop_cycle=(cycler('color', ['k','b','g','r','c','m','y'])))
p.rcParams['lines.linewidth']= 1.5
p.rcParams['axes.linewidth']=2.0

p.rcParams['text.usetex']= True
p.rcParams['mathtext.default']= 'rm'
p.rcParams['axes.formatter.use_mathtext']=False
p.rcParams['axes.labelsize']=16.0
p.rcParams['axes.unicode_minus']=False


p.rcParams['axes.linewidth']=2.5
#p.rcParams['axes.labelsize']='medium'
p.rcParams['axes.titlesize']=20#'large'
#p.rcParams['xtick.labelsize'] = 'medium' # fontsize of the tick labels
#p.rcParams['ytick.labelsize']='medium'
p.rcParams['xtick.labelsize'] = 20#'x-large' # fontsize of the tick labels
p.rcParams['ytick.labelsize']=20 #'x-large'

Lets begin by plotting reading in the final spectral table and making code to plot it

In [6]:
H = Table.read('/Users/khawkins/Desktop/data/HETDEX/HETVIPS_star_spectra_dec12_gaia.fits')

In [7]:
p.figure()
p.hexbin(H['gmag'],H['sn'], bins='log',mincnt=1)
p.axhline(y=5,ls='--',color='gray',lw=2)
p.xlabel('g') ; p.ylabel('SNR (frm Greg)')
p.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now lets measure the SNR in the same way as Hawkins+2021

In [8]:
H.colnames

['ObjID',
 'ra',
 'dec',
 'sn',
 'gmag',
 'filename',
 'index_in_file',
 'spec',
 'spec_error',
 'spec_weight',
 'RV',
 'barycorr',
 'DR3Name',
 'RAdeg',
 'DEdeg',
 'errHalfMaj',
 'errHalfMin',
 'errPosAng',
 'SolID',
 'Source',
 'RandomI',
 'e_RAdeg',
 'e_DEdeg',
 'Plx',
 'e_Plx',
 'RPlx',
 'PM',
 'pmRA',
 'e_pmRA',
 'pmDE',
 'e_pmDE',
 'RADEcor',
 'RAPlxcor',
 'RApmRAcor',
 'RApmDEcor',
 'DEPlxcor',
 'DEpmRAcor',
 'DEpmDEcor',
 'PlxpmRAcor',
 'PlxpmDEcor',
 'pmRApmDEcor',
 'NAL',
 'NAC',
 'NgAL',
 'NbAL',
 'gofAL',
 'chi2AL',
 'epsi',
 'sepsi',
 'Solved',
 'APF',
 'nueff',
 'pscol',
 'e_pscol',
 'RApscolCorr',
 'DEpscolCorr',
 'PlxpscolCorr',
 'pmRApscolCorr',
 'pmDEpscolCorr',
 'MatchObsA',
 'Nper',
 'amax',
 'MatchObs',
 'IPDgofha',
 'IPDgofhp',
 'IPDfmp',
 'IPDfow',
 'RUWE',
 'Dup',
 'o_Gmag',
 'FG',
 'e_FG',
 'RFG',
 'Gmag_x',
 'e_Gmag',
 'o_BPmag',
 'FBP',
 'e_FBP',
 'RFBP',
 'BPmag',
 'e_BPmag',
 'o_RPmag',
 'FRP',
 'e_FRP',
 'RFRP',
 'RPmag',
 'e_RPmag',
 'E(BP/RP)',
 'NBPcont

In [9]:
def add_SNR_usingfluxerr_totable(M,weight_cut = 0.5):
    per_weight_overlim = []
    SNRs = []
    M['SNR_HETDEX'] = [nan]*len(M)
    M['percent_over_weight_%.2f'%weight_cut] = [nan]*len(M)
    reordering_index= []
    for i in range(len(M)):
        weights = array(M['spec_weight'][i])
        err =  array(M['spec_error'][i])
        flux = array(M['spec'][i])
        SNRall = flux/err

        if nanmedian(SNRall) < 0:
            M['SNR_HETDEX'][i] = nan
        else:
            M['SNR_HETDEX'][i] = nanmedian(SNRall)
            M['percent_over_weight_%.2f'%weight_cut][i] = len(where(weights >= weight_cut)[0])/ float(len(weights))
    p.figure()
    ok = where(M['SNR_HETDEX'] >= 0.5)[0]
    print(nanmedian(M['SNR_HETDEX']), len(ok))
    s = p.hexbin(M['gmag'][ok], M['SNR_HETDEX'][ok],bins='log',mincnt=1E-2,cmap='plasma',yscale='log')
    cb = p.colorbar(s) ; cb.set_label('log (N)')
    p.axhline(y=5,ls='--',color='k',lw=2)
    #p.yscale('log')
    p.xlabel('G')
    p.ylabel('SNR (pixel$^{-1}$)')
    p.tight_layout()
    return M 

In [10]:
H = add_SNR_usingfluxerr_totable(H)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/khawkins/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:3823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  part.partition(kth)


12.400279634627651 224008


In [18]:
p.figure()
p.plot(H['spec'][224089])
nanmedian(array(H['spec'][224089])/array(H['spec_error'][224089]))
p.xlabel('$\lambda$/\AA'); p.ylabel('flux')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'flux')

Now lets create a code to plot up a spectrum 

In [12]:
def plot_spec(D, index=10, fig = 1,**kwargs):
    setfig(fig, figsize=(15,4))
    #p.figure(figsize=(15,4))
    p.plot(waves, D['spec'][index],**kwargs)
    p.xlabel('wavelength (\AA)')
    p.title(D['ObjID'][index])
    p.xlim([3450,5550])
    p.ylabel('Flux')
    p.tight_layout()

Lets plot a random spectrum that has good sn

In [13]:
okSNR = where(H['sn']> 20)[0]
plot_spec(H[okSNR], index=40)

Ok lets try something more complex... lets try to code up the HRD plot with the stacked spectra. This may be a challenge because we need to first do a gaia cross match

In [14]:
def make_HRD(T=None,parallax_cut=0.3, SNRcut=5,weight_lim=0.3,specpath = '/Users/khawkins/Desktop/data/HETDEX/HDR2.1/'):
    from matplotlib import cm
    import matplotlib
    #T = Table.read(HETDEX_path+'HDR2_gaia_unique_angDsort_GZ.fits')
    if T is None:
        T = Table.read(HETDEX_path+'HDR2.1/HDR2.1_Gaia_SNRs_all.fits')
    ok = where((T['e_Plx']/T['Plx'] < parallax_cut) & (T['BP-RP'] < 3.2))[0]
    T = T[ok]
    absG = T['Gmag_x'] - 5*(log10(1000./T['Plx']) - 1)
    G = T['Gmag_x']
    BPRP = T['BP-RP']
    SNR = T['sn']
    norm = matplotlib.colors.Normalize(vmin=0, vmax=10)

    b1 = where((absG >= 11.5) & (BPRP > 1.5)&(SNR > SNRcut))[0]
    b2 = where((absG > 9.8) &(absG < 11.5)& (BPRP > 1.5)&(SNR > SNRcut))[0]
    b3 = where((absG > 8.5) &(absG <= 9.8)& (BPRP > 1.5)&(SNR > SNRcut))[0]
    b4 = where((absG > 7.5) &(absG <= 8.5 )& (SNR > SNRcut))[0]
    b5 = where((absG > 6.5) &(absG <= 7.5 )& (SNR > SNRcut))[0]
    b6 = where((absG > 5.5) &(absG <= 6.5 )& (SNR > SNRcut))[0]
    b7 = where((absG > 4.5) &(absG <= 5.5 )& (BPRP >0.5)& (SNR > SNRcut) )[0]
    b8 = where((absG > 3.5) &(absG <= 4.5 )&  (BPRP >0.5)&(SNR > SNRcut))[0]
    b9 = where((absG > 2.5) &(absG <= 3.5 )&  (BPRP >0.5)&(SNR > SNRcut))[0]
    b9 = where((absG > 2.5) &(absG <= 3.5 )& (BPRP >0.5)& (SNR > SNRcut+10))[0]

    # -- def white dwarf line---
    def lin(x,m,b):
        return m*x+b
    m = (10.67-12.78)/ (-0.13-0.38)
    b= 10.99
    Ys = lin(BPRP,m,b)
    b10 = where((absG>11.0)&(SNR > SNRcut) & (BPRP <= 0.4) &(Ys-absG<0))[0]
    print(len(b10))
    bins = [b10,b9,b8,b7,b6,b5,b4,b3,b2,b1]
    WD = T[b10]
    #return WD




    p.figure(figsize=(12,7))
    ax1 = p.subplot(1,2,1)
    p.plot(BPRP, absG, 'o', color='gray',ms=0.6)
    for i in range(len(bins)):
        color = cm.plasma(norm(i),bytes=False) 

        p.plot(BPRP[bins[i]],absG[bins[i]],'o',ms=2,color=color)

    p.xlabel('(BP-RP)') ; p.ylabel(r'M$_{\mathrm{G}}$')
    p.gca().invert_yaxis()
    p.axvline(x=0.8, ls='--',color='k',lw=0.9)
    p.axvline(x=2.0, ls='--',color='k',lw=0.9)
    #p.plot(BPRP,m*BPRP+b)
    #raise a


    ax2 = p.subplot(1,2,2)
    #offset = [0,0.5,1.0,1.5,2.0,2.5,2.8,3.1,3.4,3.7]
    minF=0; offset=0
    #p.figure()
    for i in range(len(bins)):
        print(i)
        b = bins[i]
        specs = []
        color = cm.plasma(norm(i),bytes=False) 

        for j in range(len(b)):
            #if j > 10:
            #	continue
            badind = where(T['spec_weight'][b[j]] < weight_lim)[0]
            flux = T['spec'][b[j]]
            flux[badind] = nan
            specs.append(flux)
            waves = linspace(3470,5540,num=len(specs[0]))
            #p.plot(waves,flux/nanmax(flux),color='gray',lw=0.1)
            #p.plot(waves,nanmedian(specs,axis=1),'k-')


        #raise a
        waves = linspace(3470,5540,num=len(specs[0]))
        F = nanmedian(specs,axis=0)
        okwave = where((waves > 3540) & (waves < 5500))[0]
        F = F[okwave] ; waves=waves[okwave]
        normF = F/nanmax(F)
        if i < 5:
            offset=  - 0.2- (minF - nanmax(normF)) 
        else:
            offset=  - 0.55- (minF - nanmax(normF)) 

        if i==0 : offset= 0
        normF -= 0.6*i
        print(offset, minF, nanmin(normF))
        minF = nanmin(normF) 
        print('bin %i has %i stars'%(i+1,len(specs)))


        p.plot(waves, normF, color=color, lw=2.5)
        #raise a

    p.ylabel(r'F$_{\lambda}$+ cnst')
    p.gca().axes.yaxis.set_ticklabels([])
    p.xlabel('Wavelength (\AA)')

    p.tight_layout



Lets add some code to pre-process the spectra by filling nans and flux normalizing using Boosted Median

In [15]:
make_HRD(H)

62


<ipython-input-14-dcfe6a5a854d>:9: RuntimeWarning: invalid value encountered in log10
  absG = T['Gmag_x'] - 5*(log10(1000./T['Plx']) - 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/khawkins/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


0
0 0 0.23863330654515402
bin 1 has 62 stars
1
0.561366693454846 0.23863330654515402 -0.4131492565138093
bin 2 has 5262 stars
2
1.2131492565138093 -0.4131492565138093 -1.009518145211996
bin 3 has 13198 stars
3
1.8095181452119962 -1.009518145211996 -1.6359941141448557
bin 4 has 16141 stars
4
2.4359941141448553 -1.6359941141448557 -2.28290145142769
bin 5 has 14729 stars
5
2.73290145142769 -2.28290145142769 -2.9221876629981454
bin 6 has 10800 stars
6
3.3721876629981455 -2.9221876629981454 -3.5422208768391763
bin 7 has 7047 stars
7
3.992220876839176 -3.5422208768391763 -4.147665206680893
bin 8 has 5274 stars
8
4.597665206680893 -4.147665206680893 -4.75871977456819
bin 9 has 2737 stars
9
5.20871977456819 -4.75871977456819 -5.324547971940242
bin 10 has 4316 stars


Lets use topcat and crossmatch LAMOSTxGaia with HETVIPS. This table is 6,230 or so stars. Lets try to build a training set from these.

In [12]:
D = Table.read('/Users/khawkins/Desktop/data/HETDEX/HETVIPS_specxLAMOSTxGaiaDR3.fits')
D = add_SNR_usingfluxerr_totable(D)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

133.76629819320752 6230


/Users/khawkins/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [13]:
ok = where( (D['SNR_HETDEX']>10) & (D['SNR_G']>10)& (D['TEFF']>4000)& (D['TEFF']<7000)& (D['LOGG']>3.5)& \
           (D['LOGG']<5)& (D['TEFF_ERR']<100)& (D['LOGG_ERR']<0.2)& (D['FEH_ERR']<0.1)& \
           (D['TEFF']>4000) &  (D['LOGG_ERR']<0.2))[0]

In [14]:
D.colnames

['ObjID',
 'ra',
 'dec',
 'sn',
 'gmag',
 'filename_1',
 'index_in_file',
 'spec',
 'spec_error',
 'spec_weight',
 'RV_1',
 'barycorr',
 'SPECID_1',
 'RA_1',
 'DEC_1',
 'SNR_U',
 'SNR_G',
 'SNR_R',
 'SNR_I',
 'SNR_Z',
 'RV_2',
 'RV_ERR',
 'TEFF',
 'TEFF_ERR',
 'TEFF_FLAG',
 'TEFF_GRADCORR',
 'LOGG',
 'LOGG_ERR',
 'LOGG_FLAG',
 'LOGG_GRADCORR',
 'VMIC',
 'VMIC_ERR',
 'VMIC_FLAG',
 'VMIC_GRADCORR',
 'FEH',
 'FEH_ERR',
 'FEH_FLAG',
 'FEH_GRADCORR',
 'ALPHA_FE',
 'ALPHA_FE_ERR',
 'C_FE',
 'C_FE_ERR',
 'C_FE_FLAG',
 'C_FE_GRADCORR',
 'N_FE',
 'N_FE_ERR',
 'N_FE_FLAG',
 'N_FE_GRADCORR',
 'O_FE',
 'O_FE_ERR',
 'O_FE_FLAG',
 'O_FE_GRADCORR',
 'NA_FE',
 'NA_FE_ERR',
 'NA_FE_FLAG',
 'NA_FE_GRADCORR',
 'MG_FE',
 'MG_FE_ERR',
 'MG_FE_FLAG',
 'MG_FE_GRADCORR',
 'AL_FE',
 'AL_FE_ERR',
 'AL_FE_FLAG',
 'AL_FE_GRADCORR',
 'SI_FE',
 'SI_FE_ERR',
 'SI_FE_FLAG',
 'SI_FE_GRADCORR',
 'CA_FE',
 'CA_FE_ERR',
 'CA_FE_FLAG',
 'CA_FE_GRADCORR',
 'TI_FE',
 'TI_FE_ERR',
 'TI_FE_FLAG',
 'TI_FE_GRADCORR',
 'CR_FE',


In [15]:
p.figure()
S = p.hexbin(D['TEFF'][ok],D['LOGG'][ok],C=D['FEH'][ok],mincnt=1,gridsize=20)
p.gca().invert_yaxis()
p.gca().invert_xaxis()
p.xlabel('Teff (K)'); p.ylabel('logg')
p.colorbar(S,label='[Fe/H]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
p.figure()
p.plot(D['spec'][10])
p.plot(D['spec'][255],'r-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
len(ok)

2987

In [18]:

where(D['RV_1']>10)

(array([  45,  119,  225,  249,  255,  259,  264,  265,  269,  288,  291,
         292,  296,  297,  349,  357,  378,  381,  425,  440,  447,  449,
         464,  465,  467,  469,  472,  499,  526,  527,  537,  558,  623,
         640,  641,  643,  663,  675,  712,  715,  719,  730,  732,  776,
         897,  956,  966,  972,  980, 1001, 1016, 1019, 1031, 1089, 1101,
        1106, 1147, 1175, 1186, 1193, 1227, 1232, 1296, 1313, 1315, 1335,
        1358, 1379, 1394, 1403, 1406, 1419, 1451, 1499, 1516, 1531, 1555,
        1571, 1673, 1674, 1678, 1735, 1747, 1759, 1764, 1765, 1797, 1811,
        2087, 2109, 2111, 2113, 2151, 2165, 2168, 2184, 2191, 2198, 2199,
        2206, 2262, 2263, 2272, 2273, 2274, 2281, 2295, 2344, 2386, 2387,
        2388, 2391, 2393, 2394, 2400, 2409, 2442, 2575, 2577, 2581, 2673,
        2765, 2861, 2868, 2941, 2943, 2945, 2946, 2979, 2999, 3011, 3058,
        3128, 3142, 3143, 3160, 3164, 3170, 3234, 3350, 3367, 3372, 3380,
        3403, 3433, 3448, 3450, 3488, 

In [1]:
def fill_nan_21(wave, fluxes,clean_outlier=None,RV=None):
    from tqdm import tqdm
    import numpy as np
    from scipy import interpolate
    import astropy.stats
    fluxes= array(fluxes)
    inds = np.arange(fluxes.shape[0])
    
    newspec = []
    for i in tqdm(range(len(fluxes ))):
        if clean_outlier is not None:
            fluxes[i] = astropy.stats.sigma_clip(fluxes[i],sigma_upper=clean_outlier,sigma_lower=10,maxiters=2)
            #fluxes[i]=fluxes[i].filled(fill_value=nan)
        #if RV is not None:
        #	factor = RV[i] / 2.99792458E5
        #	waves = sqrt((1+factor)/(1-factor))*waves

        #good = np.where(np.isfinite(fluxes[i]))[0]
        good = where(isnan(fluxes[i]) == False)[0]
        #wavelim=[3600,5400]
        wavenew = linspace(min(wave)+100,max(wave)-100,num=len(fluxes[i])-(100) )
        #wavenew = wave

        #print shape(waves), shape(good),shape(fluxes[i]), shape(waves[good]), shape(fluxes[good])
        #print good, fluxes[i][good],i 
        f = interpolate.interp1d(wave[good], fluxes[i][good],"cubic", bounds_error=True)
        newspec.append(f(wavenew))
    wavenew = [linspace(min(wave),max(wave),num=len(fluxes[i]) )] * len(fluxes)
    wavenew = array(wavenew) ; newspec=array(newspec)
    p.plot(wavenew,newspec/median(newspec))
    #B = np.where(np.isfinite(A),A,f(inds))
    return newspec



def BMCcntflux(lams, fluxes,width = 50,ptile=98, cpy = False,plt=False):
    import copy as cp
    import time
    from scipy import stats
    continuumflux = []

    for i in arange(len(fluxes)):
        testind = where((lams >= lams[i]-(width/2.0)) & (lams <= lams[i]+(width/2.0)))[0]
        continuumflux.append(stats.scoreatpercentile(fluxes[testind],ptile))

    if plt:
        p.plot(lams,fluxes)
        p.plot(lams,continuumflux,'-')
    fluxes /= continuumflux
    return fluxes

def BMCcntflux_poly(lams, fluxes,width = 20,ptile=98, cpy = False,plt=False,deg = 3):
    import numpy as np
    import copy as cp
    continuumpts = []
    lampts = []

    for i in arange(len(fluxes)):
        testind = where((lams >= lams[i]-(width/2.0)) & (lams <= lams[i]+(width/2.0)))[0]
        lampts.append( (lams[i]-(width/2.0) +lams[i]+(width/2.0) )/2.0 )
        if isnan(percentile(fluxes[testind],ptile) ) == False:
            continuumpts.append(percentile(fluxes[testind],ptile))
        else:
            continue
    try:
        Z  = np.polyfit(lampts, continuumpts,deg=deg)
    except TypeError:
        print('WARNING couldnt fit conitnuum returning nans')
        return np.array([nan]*len(lams))
    func = np.poly1d(Z)
    continuumflux = func(lams)
    if plt:
        p.plot(lams,fluxes)
        p.plot(lams,continuumflux,'-')
    fluxes /= continuumflux
    return fluxes

In [2]:
flux2 = fill_nan_21(waves,D['spec'][ok],clean_outlier=4)

NameError: name 'waves' is not defined

In [143]:
flux2

array([[         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       ...,
       [216.30177835, 272.33189463, 232.44264507, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan]])

In [138]:
Nflux = BMCcntflux(waves,flux2)

IndexError: index 1036 is out of bounds for axis 0 with size 1036

In [161]:
#ok_pix = [nan]*len(D['spec'][0])
ok_pix= []
for i in range(1036):
    if any(isnan(D['spec'][:,i])):
        continue
    else:
        ok_pix.append(i)

In [162]:
ok_pix

[]